In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import savgol_filter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt

from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import accuracy_score
import json
import re
from collections import namedtuple
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")

In [20]:
N_WAVELENGTH = 228
LOD_URE = 1000

In [21]:
df = pd.read_csv('data_NIRS/Merged_db.csv').drop_duplicates()
df.head()

,Mã NIR,Urea,Nhóm đo,Chỉ tiêu đo,Loại cá,Địa điểm lấy mẫu,STT mẫu cá,Vị trí đo,Lần lặp,Phương pháp đo,...,230,231,232,233,234,235,236,237,238,239
0,2113099a1,19147.4,Đà Nẵng,Ure,Nục 1,Cảng cá,99,"Ngoài da, gáy",1,Hadamard 1,...,-0.136101,-0.287185,-0.380416,-0.459176,-0.407652,-0.472400,-0.429911,-0.333819,-0.395731,-0.167501
1,2113099a2,19147.4,Đà Nẵng,Ure,Nục 1,Cảng cá,99,"Ngoài da, gáy",2,Hadamard 1,...,-0.135012,-0.306666,-0.467668,-0.492477,-0.481871,-0.370628,-0.297349,-0.409274,-0.486664,-0.426964
2,2113099a3,19147.4,Đà Nẵng,Ure,Nục 1,Cảng cá,99,"Ngoài da, gáy",3,Hadamard 1,...,-0.324308,-0.427586,-0.539696,-0.547557,-0.575114,-0.555003,-0.462344,-0.440240,-0.298107,-0.313747
3,2113099a4,19147.4,Đà Nẵng,Ure,Nục 1,Cảng cá,99,"Ngoài da, gáy",4,Hadamard 1,...,-0.298622,-0.377505,-0.534373,-0.577949,-0.533663,-0.570596,-0.488621,-0.419825,-0.354021,-0.295377
4,2113099a5,19147.4,Đà Nẵng,Ure,Nục 1,Cảng cá,99,"Ngoài da, gáy",5,Hadamard 1,...,-0.272732,-0.429970,-0.496265,-0.559068,-0.586745,-0.516311,-0.469121,-0.351306,-0.360435,-0.257529


# Xử lý dữ liệu NaN và vô hạn

In [22]:
print(f'Tổng các điểm dữ liệu trống trước xử lý: {df.iloc[:,-N_WAVELENGTH:].isna().sum().sum()}')
print(f'Tổng các điểm dữ liệu vô hạn trước xử lý: {np.isinf(df.iloc[:,-N_WAVELENGTH:]).sum().sum()}')

Tổng các điểm dữ liệu trống trước xử lý: 0
Tổng các điểm dữ liệu vô hạn trước xử lý: 0


In [23]:
# Lấy tất cả vị trí của các trị trống hoặc vô hạn
null_check = df.isna()
indices_null = np.argwhere(null_check.to_numpy() == True)

df_inf = df.iloc[:,-N_WAVELENGTH:]
inf_check = np.isinf(df_inf)
indices_inf = np.argwhere(inf_check.to_numpy()== True)
indices_inf[:,1] += df.shape[1] - N_WAVELENGTH

indices = np.concatenate((indices_inf, indices_null))

In [24]:
n_nearest = 2 # Sử dụng n giá trị gần nhất cả 2 phía trước và sau để tính mean của missing data
for x,y in indices:
    if y < df.shape[1] - N_WAVELENGTH: # Chỉ xử lý các missing data của phổ
        continue
    #sample = df.iloc[x, np.maximum(y - n_nearest , 0):np.minimum(y + 1 + n_nearest, df.shape[1]-1)]
    left = df.iloc[x,  np.maximum(y - n_nearest , 0) : y]
    right = df.iloc[x, y + 1 : np.minimum(y + 1 + n_nearest, df.shape[1]-1)]
    sample = left.combine_first(right)
    df.iloc[x, y] = np.mean(sample)

In [25]:
print(f'Tổng các điểm dữ liệu trống sau xử lý: {df.iloc[:,-N_WAVELENGTH:].isna().sum().sum()}')
print(f'Tổng các điểm dữ liệu vô hạn sau xử lý: {np.isinf(df.iloc[:,-N_WAVELENGTH:]).sum().sum()}')

Tổng các điểm dữ liệu trống sau xử lý: 0
Tổng các điểm dữ liệu vô hạn sau xử lý: 0


# Sửa các mã NIR bị lỗi 
![image.png](attachment:image.png)

In [27]:

configJsonLink = 'config.json'
config = json.load(open(configJsonLink, encoding="utf8"))
config = namedtuple("ObjectName", config.keys())(*config.values())

nirsMetadataCoding = json.load(open(config.nirsMetadataCodingFileName, encoding="utf8"))
nirsMetadataCoding = namedtuple("ObjectName", nirsMetadataCoding.keys())(*nirsMetadataCoding.values())

In [28]:
count = 0
for i in df.index:
    nir_code = df['Mã NIR'][i]
    if not bool(re.search(r'[a-z]',nir_code)):
        nir_code_fixed = ''
        for nirsIdComponent in config.nirsIdComponents:
            nirsMetadata = [data for data in nirsMetadataCoding.nirsIdComponents if data['name'] == nirsIdComponent['name']][0]
            
            for value in nirsMetadata['decode']:
                name = nirsIdComponent['name']
                if (df[name][i] == value['name']):
                    if name == 'STT mẫu cá':
                        nir_code_fixed += str(value['key']).zfill(3)
                    else:
                        nir_code_fixed += str(value['key'])
                        
        print(f'{i} : {nir_code} ==> {nir_code_fixed}')
        df['Mã NIR'][i] = nir_code_fixed
        count +=1
        
print('samples replaced: ', count)

20 : 2.11E+07 ==> 2113099er1
21 : 2.11E+08 ==> 2113099er2
22 : 2.11E+09 ==> 2113099er3
23 : 2.11E+10 ==> 2113099er4
24 : 2.11E+11 ==> 2113099er5
80 : 2.11E+07 ==> 2113100er1
81 : 2.11E+08 ==> 2113100er2
82 : 2.11E+09 ==> 2113100er3
83 : 2.11E+10 ==> 2113100er4
84 : 2.11E+11 ==> 2113100er5
140 : 2.11E+07 ==> 2113101er1
141 : 2.11E+08 ==> 2113101er2
142 : 2.11E+09 ==> 2113101er3
143 : 2.11E+10 ==> 2113101er4
144 : 2.11E+11 ==> 2113101er5
200 : 2.11E+07 ==> 2113113er1
201 : 2.11E+08 ==> 2113113er2
202 : 2.11E+09 ==> 2113113er3
203 : 2.11E+10 ==> 2113113er4
204 : 2.11E+11 ==> 2113113er5
260 : 2.11E+07 ==> 2113114er1
261 : 2.11E+08 ==> 2113114er2
262 : 2.11E+09 ==> 2113114er3
263 : 2.11E+10 ==> 2113114er4
264 : 2.11E+11 ==> 2113114er5
320 : 2.11E+07 ==> 2113115er1
321 : 2.11E+08 ==> 2113115er2
322 : 2.11E+09 ==> 2113115er3
323 : 2.11E+10 ==> 2113115er4
324 : 2.11E+11 ==> 2113115er5
380 : 2.12E+07 ==> 2123188er1
381 : 2.12E+08 ==> 2123188er2
382 : 2.12E+09 ==> 2123188er3
383 : 2.12E+10 ==> 2

# Tạo test data từ dữ liệu gốc, tất cả các mẫu NIR của 1 con cá (thường là 60) thì sẽ nằm trong test data

In [29]:
def get_random_test_fish_id(df):
    TEST_SIZE = 0.2

    fish_numbers = df['STT mẫu cá'].unique()

    ure_value = []
    for number in fish_numbers:
        sub_df = df[df['STT mẫu cá'] == number]
        # print(stt, sub_df['Urea'].value_counts())
        ure_value.append(sub_df['Urea'].iloc[1])
    y_labels = np.where(np.array(ure_value) >= LOD_URE, 1, 0)

    number_test_fish = int(len(fish_numbers)*TEST_SIZE)

    indices_below_LOD = np.where(y_labels == 0)[0]
    test_indices_below_LOD = np.random.choice(indices_below_LOD, size=number_test_fish//2, replace=False)
    test_fish_numbers_below_LOD = fish_numbers[test_indices_below_LOD]

    indices_above_LOD = np.where(y_labels == 1)[0]
    test_indices_above_LOD = np.random.choice(indices_above_LOD, size=number_test_fish//2, replace=False)
    test_fish_numbers_above_LOD = fish_numbers[test_indices_above_LOD]

    test_fish_number = np.hstack((test_fish_numbers_above_LOD, test_fish_numbers_below_LOD))
    
    return test_fish_number

def train_smote_and_test_by_test_fish_id(df, test_id_fish:list):
    test_data = pd.DataFrame()
    train_data = df.copy()
    for i in test_id_fish:
        sub_df = df[df['STT mẫu cá'] == i]
        test_data = pd.concat([test_data, sub_df], axis=0)
        
        train_data = train_data.drop(train_data[train_data['STT mẫu cá'] == i].index)
        
    return train_data, test_data

def my_split_train_test(train_data, test_data):
    x_train = train_data.iloc[:,-N_WAVELENGTH:]
    y_train = np.where(train_data['Urea'] >= LOD_URE, 1, 0)
    over_sample = SMOTE()
    x_train_smote, y_train_smote = over_sample.fit_resample(x_train, y_train)
    
    x_test = test_data.iloc[:,-N_WAVELENGTH:]
    y_test = np.where(test_data['Urea'] >= LOD_URE, 1, 0)
    
    return x_train_smote, y_train_smote, x_test, y_test

In [129]:

test_fish_id = get_random_test_fish_id(df)
train_data, test_data = train_smote_and_test_by_test_fish_id(df, test_fish_id)

X_TRAIN = train_data.iloc[:,-N_WAVELENGTH:]
Y_TRAIN = train_data['Urea'] 
Y_TRAIN_LOG = np.log10(Y_TRAIN)

X_TEST = test_data.iloc[:,-N_WAVELENGTH:]
Y_TEST = test_data['Urea']
Y_TEST_LOG = np.log10(Y_TEST)

Y_TRAIN_SNV_LOG = (Y_TRAIN_LOG - Y_TRAIN_LOG.mean()) / Y_TRAIN_LOG.std(ddof=0)
Y_TEST_SNV_LOG = (Y_TEST_LOG - Y_TEST_LOG.mean()) / Y_TEST_LOG.std(ddof=0)

Y_TRAIN_SNV = (Y_TRAIN - Y_TRAIN.mean()) / Y_TRAIN.std(ddof=0)
Y_TEST_SNV = (Y_TEST - Y_TEST.mean()) / Y_TEST.std(ddof=0)

print(f'{Y_TRAIN_SNV.mean()} + {Y_TRAIN_SNV.std(ddof=0)}')
print(f'{Y_TEST_SNV.mean()} + {Y_TEST_SNV.std(ddof=0)}')

print(f'{Y_TRAIN.mean()} + {Y_TRAIN.std(ddof=0)}')
print(f'{Y_TEST.mean()} + {Y_TEST.std(ddof=0)}')

4.1981845539740043e-17 + 0.9999999999999999
3.3835368369528583e-17 + 1.0
1105.9931663842244 + 4658.433914079745
1660.7993481803571 + 3517.4412139962856


In [134]:
print(len(Y_TRAIN))
print(len(Y_TEST))

6770
1680


# Tiền xử lý

Savizky-Golay 

In [110]:
def my_SG(X, deriv=0):
    X_SG = savgol_filter(X, window_length=25, polyorder=5, deriv=deriv)
    return pd.DataFrame(X_SG)

In [49]:
def snv(df_input, N_WAVELENGTH=228):
    df_input = pd.DataFrame(df_input)
    df_output = df_input.copy()
    for i in range(df_output.shape[0]):
        temp_nir = []
        temp_nir = df_output.iloc[i,-N_WAVELENGTH:].to_numpy()
        temp_mean = temp_nir.mean()
        temp_std = temp_nir.std(ddof=0)
        df_output.iloc[i,-N_WAVELENGTH:] = (df_output.iloc[i,-N_WAVELENGTH:] - temp_mean) / temp_std
    return df_output

def msc(df_input, N_WAVELENGTH=228):
    df_input = pd.DataFrame(df_input)
    df_output = df_input.copy()
    df_input = (df_input.T - np.mean(df_input.T, axis=0)).T
    ref = np.mean(df_input, axis=0)

    for i in range(df_input.shape[0]):
        # Run regression
        fit = np.polyfit(ref, df_input.iloc[i,:], 1, full=True)
        # Apply correction
        df_output.iloc[i,-N_WAVELENGTH:] = (df_input.iloc[i,:] - fit[0][1]) / fit[0][0] 
    return df_output

def my_MinMaxScaler(df_input, N_WAVELENGTH=228):
    df_input = pd.DataFrame(df_input)
    df_output = df_input.copy()
    for i in range(df_output.shape[0]):
        x_min = min(df_output.iloc[i,-N_WAVELENGTH:])
        x_max = max(df_output.iloc[i,-N_WAVELENGTH:])
        df_output.iloc[i,-228:] = (df_output.iloc[i,-N_WAVELENGTH:] - x_min) / (x_max - x_min)
    return df_output

def my_RobustScaler(df_input, N_WAVELENGTH=228):
    df_input = pd.DataFrame(df_input)
    df_output = df_input.copy()
    for i in range(df_output.shape[0]):
        temp_nir = []
        temp_nir = df_output.iloc[i,-N_WAVELENGTH:].to_numpy()
        Q1 = np.percentile(temp_nir, 25)
        median = np.percentile(temp_nir, 50)
        Q3 = np.percentile(temp_nir, 75)
        df_output.iloc[i,-228:] = (df_output.iloc[i,-N_WAVELENGTH:] - median) / (Q3 - Q1)
    return df_output


In [51]:
def my_preprocess_data(X_data, feature='sg0', normalization = 'msc'):
    X_data_prep = X_data

    if normalization == 'msc':
        X_data_prep = msc(X_data)
    elif normalization == 'snv':
        X_data_prep = snv(X_data)
    elif normalization == 'minmax':
        X_data_prep = my_MinMaxScaler(X_data)
    elif normalization == 'robust':
        X_data_prep = my_RobustScaler(X_data)
        
    if feature == 'sg0':
        X_data_prep = my_SG(X_data_prep, 0)
    elif feature == 'sg1':
        X_data_prep = my_SG(X_data_prep, 1)
    elif feature == 'sg2':
        X_data_prep = my_SG(X_data_prep, 2)
    elif feature == 'sg1 + sg2':
        X_data_SG1 = my_SG(X_data_prep, 1)
        X_data_SG2 = my_SG(X_data_prep, 2)
        X_data_prep = np.hstack([X_data_SG1, X_data_SG2])
        
    return X_data_prep

# PLSR

In [132]:
best_nc = 0
max_r2 = -1

for nc in range(3,60):
    pls = PLSRegression(n_components=nc)
    pls.fit(X_TRAIN, Y_TRAIN_SNV)
    y_pred = pls.predict(X_TEST)
    r2 = metrics.r2_score(Y_TEST_SNV, y_pred)
    if r2 > max_r2:
        max_r2 = r2
        best_nc = nc

print(best_nc, max_r2)

35 0.15970355871191477


In [101]:
def my_plsr(X_train, y_train, X_test, y_test):
    pls = PLSRegression(n_components=best_nc)
    pls.fit(X_train, y_train)
    y_pred = pls.predict(X_test)
    mse = metrics.mean_squared_error(y_true=y_test, y_pred=y_pred)
    r2 = metrics.r2_score(y_true=y_test, y_pred=y_pred, force_finite=False)
    return mse, r2

In [131]:
all_feature_exaction = ['sg0', 'sg1', 'sg2', 'sg1 + sg2']
all_normalization = ['msc','snv', 'minmax', 'robust']

list_msc = []    
list_r2 = []  
for feature in all_feature_exaction:
    for normal in all_normalization:

        X_train = my_preprocess_data(X_TRAIN, feature=feature, normalization=normal)
        X_test = my_preprocess_data(X_TEST, feature=feature, normalization=normal)
        
        mse, r2 = my_plsr(X_train, Y_TRAIN_SNV, X_test, Y_TEST_SNV)
        list_msc.append(mse)
        list_r2.append(r2)
        print(mse, r2)

0.8720318031391862 0.12796819686081395
0.8338566675921477 0.1661433324078524
0.8172221038499775 0.1827778961500226
0.8182274654103732 0.18177253458962694
0.8730671322746213 0.12693286772537882
0.8413904731100877 0.15860952688991237
0.8230357580977729 0.17696424190222726
0.8123598087127049 0.18764019128729525
0.863067544859851 0.13693245514014907
0.8371468086155933 0.1628531913844068
0.8116829566833227 0.18831704331667742
0.8188892096961516 0.18111079030384847
0.8683225496163749 0.13167745038362522
0.8431974547919723 0.15680254520802783
0.8214724906271388 0.17852750937286133
0.8168467632361953 0.1831532367638048


In [133]:
print(min(list_msc), np.argmin(list_msc), max(list_r2), np.argmax(list_r2))

0.8116829566833227 10 0.18831704331667742 10


=> sử dụng đặc trưng x_sg0_minmax cho kết quả tối nhất

In [59]:
for position in list(test_data['Vị trí đo'].unique()):
    sub_data_TEST = test_data[test_data['Vị trí đo'] == position]
    sub_X_TEST = sub_data_TEST.iloc[:, -N_WAVELENGTH:]
    sub_Y_TEST = sub_data_TEST['Urea']

    X_train_pred = my_preprocess_data(X_TRAIN, feature='sg0', normalization='minmax')
    sub_X_test_pred = my_preprocess_data(sub_X_TEST, feature='sg0', normalization='minmax')
    sub_Y_TEST_pred = (sub_Y_TEST - sub_Y_TEST.mean()) / sub_Y_TEST.std(ddof=0)
    
    mse, r2 = my_plsr(X_train_pred, Y_TRAIN_SNV, sub_X_test_pred, sub_Y_TEST_pred)
    
    print(f'Vị trí : {position}, MSE : {mse}  ==>  R2: {r2}') 


Vị trí : Ngoài da, gáy, MSE : 0.6731257345240262  ==>  R2: 0.32687426547597354
Vị trí : Ngoài da, lưng, MSE : 0.6467652978702059  ==>  R2: 0.3532347021297939
Vị trí : Ngoài da, đuôi, MSE : 0.6445933473056358  ==>  R2: 0.355406652694364
Vị trí : Ngoài da, bụng, MSE : 0.6342747689659973  ==>  R2: 0.36572523103400245
Vị trí : Mang 1, MSE : 0.7807751642436356  ==>  R2: 0.2192248357563641
Vị trí : Mang 2, MSE : 0.7706922983707807  ==>  R2: 0.22930770162921899
Vị trí : Mắt 1, MSE : 0.8144906812154761  ==>  R2: 0.1855093187845236
Vị trí : Mắt 2, MSE : 1.0302090637052472  ==>  R2: -0.030209063705247674
Vị trí : Trong thịt, gáy, MSE : 0.8608617801752967  ==>  R2: 0.13913821982470298
Vị trí : Trong thịt, lưng, MSE : 0.839178766492066  ==>  R2: 0.16082123350793365
Vị trí : Trong thịt, đuôi, MSE : 0.7923085610746283  ==>  R2: 0.20769143892537134
Vị trí : Trong thịt, bụng, MSE : 0.7749519983964578  ==>  R2: 0.22504800160354188


In [52]:
list_msc_2 = []    
list_r2_2 = []  
for feature in all_feature_exaction:
    for normal in all_normalization:

        X_train = my_preprocess_data(X_TRAIN.iloc[:,n_optimal_wavelength_plsr], feature=feature, normalization=normal)
        X_test = my_preprocess_data(X_TEST.iloc[:,n_optimal_wavelength_plsr], feature=feature, normalization=normal)
        
        mse, r2 = my_plsr(X_train, Y_TRAIN_LOG_SNV, X_test, Y_TEST_LOG_SNV)
        list_msc_2.append(mse)
        list_r2_2.append(r2)
        print(mse, r2)

9.572188535832918 -8.572188535832918
1.5090519392165171 -0.5090519392165171
1.1788812122745063 -0.17888121227450626
3.727564293365865 -2.727564293365865
4.771589462170817 -3.771589462170817
46.5491084360717 -45.5491084360717
290.62435341756645 -289.62435341756645
7.91090859349336 -6.91090859349336
85342.66612672563 -85341.66612672563
7293.098481023015 -7292.098481023015
3710.712748117322 -3709.712748117322
643.652963574902 -642.652963574902
0.9559000125919348 0.0440999874080652
0.9030378800306523 0.09696211996934767
0.8519113652672526 0.14808863473274736
0.8425743401538799 0.15742565984612011


In [54]:
print(min(list_msc_2), np.argmin(list_msc_2), max(list_r2_2), np.argmax(list_r2_2))

0.8425743401538799 15 0.15742565984612011 15


# Cross validation

In [55]:
from sklearn.model_selection import StratifiedKFold
K_FOLD = 5
skf = StratifiedKFold(n_splits=K_FOLD, shuffle=True, random_state=1)
TEST_SIZE = 0.2

fish_numbers = df['STT mẫu cá'].unique()

ure_value = []
for number in fish_numbers:
    sub_df = df[df['STT mẫu cá'] == number]
    # print(stt, sub_df['Urea'].value_counts())
    ure_value.append(sub_df['Urea'].iloc[1])
    
y_labels = np.where(np.array(ure_value) >= LOD_URE, 1, 0)

In [71]:
result_cv_position = {}

for times, (train_index, test_index) in enumerate(skf.split(ure_value, y_labels)):
    mse_each_position = []
    r2_each_position = []
    
    test_fish_id = fish_numbers[test_index]
    train_data, test_data = train_smote_and_test_by_test_fish_id(df, test_fish_id)
    x_train = train_data.iloc[:,-N_WAVELENGTH:]
    y_train = np.log10(train_data['Urea'])
    
    for position in list(test_data['Vị trí đo'].unique()):
        sub_data_TEST = test_data[test_data['Vị trí đo'] == position]
        sub_X_TEST = sub_data_TEST.iloc[:, -N_WAVELENGTH:]
        sub_Y_TEST = np.log10(sub_data_TEST['Urea'])
   
        X_train_pred = my_preprocess_data(x_train, feature='sg0', normalization='minmax')
        sub_X_test_pred = my_preprocess_data(sub_X_TEST, feature='sg0', normalization='minmax')
        
        y_train_prep = (y_train - y_train.mean()) / y_train.std(ddof=0)
        sub_y_test_prep = (sub_Y_TEST - sub_Y_TEST.mean()) / sub_Y_TEST.std(ddof=0)
        
        mse, r2 = my_plsr(X_train_pred, y_train_prep, sub_X_test_pred, sub_y_test_prep)
        
        print(f'Lần: {times+1},Vị trí : {position}, MSE : {mse}  ==>  R2: {r2}') 
        mse_each_position.append(mse)
        r2_each_position.append(r2)
        
    result_cv_position[times] = (mse_each_position, r2_each_position)
    print('='*30)

Lần: 1,Vị trí : Ngoài da, gáy, MSE : 0.9263829146135578  ==>  R2: 0.07361708538644252
Lần: 1,Vị trí : Ngoài da, lưng, MSE : 0.8077639230824746  ==>  R2: 0.19223607691752576
Lần: 1,Vị trí : Ngoài da, đuôi, MSE : 0.7264901758418508  ==>  R2: 0.2735098241581494
Lần: 1,Vị trí : Ngoài da, bụng, MSE : 0.7500868752284916  ==>  R2: 0.24991312477150862
Lần: 1,Vị trí : Mang 1, MSE : 0.9908153255718228  ==>  R2: 0.009184674428177497
Lần: 1,Vị trí : Mang 2, MSE : 0.9848559269740702  ==>  R2: 0.015144073025930238
Lần: 1,Vị trí : Mắt 1, MSE : 1.0759655731601705  ==>  R2: -0.07596557316017005
Lần: 1,Vị trí : Mắt 2, MSE : 1.0021488875572968  ==>  R2: -0.002148887557296364
Lần: 1,Vị trí : Trong thịt, gáy, MSE : 0.9468549404346464  ==>  R2: 0.05314505956535409
Lần: 1,Vị trí : Trong thịt, lưng, MSE : 1.01390333672825  ==>  R2: -0.013903336728249727
Lần: 1,Vị trí : Trong thịt, đuôi, MSE : 0.892577033307275  ==>  R2: 0.10742296669272522
Lần: 1,Vị trí : Trong thịt, bụng, MSE : 0.8936664117229047  ==>  R2: 0

In [82]:
for i, position in enumerate(list(df['Vị trí đo'].unique())):
    sub_list_mse = []
    for j in range(K_FOLD):
        mse = result_cv_position[j][0][i]
        sub_list_mse.append(mse)
        
    sub_list_r2 = []
    for j in range(K_FOLD):
        r2 = result_cv_position[j][1][i]
        sub_list_r2.append(r2)
    print('Vị trí : {}  ==>  mse trung binh: {:.3f}, r2 trung binh: {:.3f}'.format(position, np.mean(sub_list_mse), np.mean(sub_list_r2)))

Vị trí : Ngoài da, gáy  ==>  mse trung binh: 0.898, r2 trung binh: 0.102
Vị trí : Ngoài da, lưng  ==>  mse trung binh: 0.847, r2 trung binh: 0.153
Vị trí : Ngoài da, đuôi  ==>  mse trung binh: 0.797, r2 trung binh: 0.203
Vị trí : Ngoài da, bụng  ==>  mse trung binh: 0.792, r2 trung binh: 0.208
Vị trí : Mang 1  ==>  mse trung binh: 0.962, r2 trung binh: 0.038
Vị trí : Mang 2  ==>  mse trung binh: 0.971, r2 trung binh: 0.029
Vị trí : Mắt 1  ==>  mse trung binh: 1.089, r2 trung binh: -0.089
Vị trí : Mắt 2  ==>  mse trung binh: 1.037, r2 trung binh: -0.037
Vị trí : Trong thịt, gáy  ==>  mse trung binh: 0.948, r2 trung binh: 0.052
Vị trí : Trong thịt, lưng  ==>  mse trung binh: 0.953, r2 trung binh: 0.047
Vị trí : Trong thịt, đuôi  ==>  mse trung binh: 0.940, r2 trung binh: 0.060
Vị trí : Trong thịt, bụng  ==>  mse trung binh: 0.900, r2 trung binh: 0.100


=====================================

In [83]:
result_cv_position2 = {}

for times, (train_index, test_index) in enumerate(skf.split(ure_value, y_labels)):
    mse_each_position = []
    r2_each_position = []
    
    test_fish_id = fish_numbers[test_index]
    train_data, test_data = train_smote_and_test_by_test_fish_id(df, test_fish_id)
    x_train = train_data.iloc[:,-N_WAVELENGTH:]
    y_train = np.log10(train_data['Urea'])
    
    for position in list(test_data['Vị trí đo'].unique()):
        sub_data_TEST = test_data[test_data['Vị trí đo'] == position]
        sub_X_TEST = sub_data_TEST.iloc[:, -N_WAVELENGTH:]
        sub_Y_TEST = np.log10(sub_data_TEST['Urea'])
   
        X_train_pred = my_preprocess_data(x_train, feature='sg0', normalization='minmax')
        sub_X_test_pred = my_preprocess_data(sub_X_TEST, feature='sg0', normalization='minmax')
        
        
        mse, r2 = my_plsr(X_train_pred, y_train, sub_X_test_pred, sub_Y_TEST)
        
        print(f'Lần: {times+1},Vị trí : {position}, MSE : {mse}  ==>  R2: {r2}') 
        mse_each_position.append(mse)
        r2_each_position.append(r2)
        
    result_cv_position2[times] = (mse_each_position, r2_each_position)
    print('='*30)

Lần: 1,Vị trí : Ngoài da, gáy, MSE : 0.4160619089504217  ==>  R2: 0.12907471463666542
Lần: 1,Vị trí : Ngoài da, lưng, MSE : 0.39838649835411577  ==>  R2: 0.16607392481751448
Lần: 1,Vị trí : Ngoài da, đuôi, MSE : 0.3657442474852368  ==>  R2: 0.23440260629810528
Lần: 1,Vị trí : Ngoài da, bụng, MSE : 0.3913263967691  ==>  R2: 0.1808525451510491
Lần: 1,Vị trí : Mang 1, MSE : 0.4864027353961197  ==>  R2: -0.018166844917425484
Lần: 1,Vị trí : Mang 2, MSE : 0.49082260839063113  ==>  R2: -0.02741878330977965
Lần: 1,Vị trí : Mắt 1, MSE : 0.5508234266560117  ==>  R2: -0.15301602892554467
Lần: 1,Vị trí : Mắt 2, MSE : 0.5053259099499101  ==>  R2: -0.05777794808189207
Lần: 1,Vị trí : Trong thịt, gáy, MSE : 0.4734594593721632  ==>  R2: 0.008926782550509715
Lần: 1,Vị trí : Trong thịt, lưng, MSE : 0.5079396873723435  ==>  R2: -0.038007701831097584
Lần: 1,Vị trí : Trong thịt, đuôi, MSE : 0.44534190183146044  ==>  R2: 0.08991493371869519
Lần: 1,Vị trí : Trong thịt, bụng, MSE : 0.4729255739702655  ==>  R

In [84]:
for i, position in enumerate(list(df['Vị trí đo'].unique())):
    sub_list_mse = []
    for j in range(K_FOLD):
        mse = result_cv_position2[j][0][i]
        sub_list_mse.append(mse)
        
    sub_list_r2 = []
    for j in range(K_FOLD):
        r2 = result_cv_position2[j][1][i]
        sub_list_r2.append(r2)
    print('Vị trí : {}  ==>  mse trung binh: {:.3f}, r2 trung binh: {:.3f}'.format(position, np.mean(sub_list_mse), np.mean(sub_list_r2)))

Vị trí : Ngoài da, gáy  ==>  mse trung binh: 0.396, r2 trung binh: 0.071
Vị trí : Ngoài da, lưng  ==>  mse trung binh: 0.378, r2 trung binh: 0.137
Vị trí : Ngoài da, đuôi  ==>  mse trung binh: 0.359, r2 trung binh: 0.175
Vị trí : Ngoài da, bụng  ==>  mse trung binh: 0.347, r2 trung binh: 0.194
Vị trí : Mang 1  ==>  mse trung binh: 0.436, r2 trung binh: -0.021
Vị trí : Mang 2  ==>  mse trung binh: 0.440, r2 trung binh: -0.019
Vị trí : Mắt 1  ==>  mse trung binh: 0.499, r2 trung binh: -0.169
Vị trí : Mắt 2  ==>  mse trung binh: 0.463, r2 trung binh: -0.093
Vị trí : Trong thịt, gáy  ==>  mse trung binh: 0.427, r2 trung binh: -0.002
Vị trí : Trong thịt, lưng  ==>  mse trung binh: 0.425, r2 trung binh: 0.021
Vị trí : Trong thịt, đuôi  ==>  mse trung binh: 0.416, r2 trung binh: 0.033
Vị trí : Trong thịt, bụng  ==>  mse trung binh: 0.417, r2 trung binh: 0.035
